In [12]:
language = 'french'
setting = "Normandie viking du 9e siècle"

In [21]:
from langchain_community.llms.mlx_pipeline import MLXPipeline
from langchain_community.chat_models.mlx import ChatMLX
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.globals import set_verbose
from langchain.globals import set_debug

#set_debug(True)
set_verbose(True)

print('Loading model...')
# Load model from huggingface, using the MLX framework to take advantage of Apple Silicon
llm = MLXPipeline.from_model_id(
    #"mlx-community/Meta-Llama-3.1-8B-Instruct-8bit",
    #"mlx-community/Meta-Llama-3.1-8B-Instruct-bf16",
    #"mlx-community/Qwen2.5-32B-Instruct-4bit", # Let's try using a larger model, see if that improves results
    "mlx-community/Llama-3.2-3B-Instruct-8bit",
    pipeline_kwargs={"max_tokens": 2048, "temp": 0.2, "repetition_penalty":1.2},
)

# Setup verbose mode: https://stackoverflow.com/a/77629872/10914628
model = ChatMLX(llm=llm).with_config({'callbacks': [ConsoleCallbackHandler()]})
print('Model loaded.')


Loading model...


/Users/frieder/Documents/GitHub/procedural-stories/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 89877.94it/s]


Model loaded.


In [1]:
from owlready2 import *


onto = get_ontology('./populated_story.owl').load()


with onto:
    for c in onto.Character.instances():
        print(c.hasName)

    print(onto.Player.instances())

Thora la Forgeronne
Ormr le Sage
Erik Stormbringer
Jarl Bjorn Ironheart
Skadi la Voyante
Sigrid the Seafarer
Gunnar le Marchand
Hilda la Cartographe
Ogmund le Chasseur
Grimgor le Rôdeur
Olav le Navigateur
Druid Alvar
Sylvain le Guetteur
Harald le Vétéran
[populated_story.Erik%20Stormbringer]


# Let's model the first turn

In [26]:
FIRST_TURN_PROMPT = """
You are an LLM designed to act as the engine for a text adventure game set in "{{setting}}".

List of the locations that make up this game's world (to give you general context):
{%- for location in locations %}
    - {{ location.hasName }}: {{location.hasDescription}}
{%- endfor %}

The player is located at {{player_start_location_name}}.
The locations accessible from where the player is are {{nearby_locations_names}}

Characters present in this location:
{%- for character in characters_nearby %}
    - {{ character.hasName }}: {{character.hasDescription}} (narrative importance {{character.importance}})
{%- endfor %}

Items present in this location:
{%- for item in items_nearby %}
    - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.importance}})
{%- endfor %}

You're about to start interacting with the player. The player plays a character in the story:
    "{{player.hasName}}": {{player.hasDescription}}

The player's goal is: {{player.hasGoal.hasDescription}}

You need to write an SHORT introduction, explaining where the player is, who they are, and give them a motivation to reach their goal.
Keep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.
Keep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large.

The game is played by interactions between the game (you) and the player. You will first produce a setting, then the player will type in natural language whatever they want to do, etc.
Do not reveal everything all at once: let the player discover things. Only output natural text, as one would read in a book they actually were the hero of.

Your entire output will be visible to the player, so stay in your role of narrator ! All your output should be in {{language}}
"""

In [53]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(
    template=FIRST_TURN_PROMPT,
    template_format='jinja2'
)

with onto:
    player = list(onto.Player.instances())[0]
    current_location = player.INDIRECT_isAtLocation
    locations = onto.Location.instances()
    characters_nearby = current_location.INDIRECT_containsCharacter
    locations_nearby = current_location.INDIRECT_isLinkedToLocation
    nearby_locations_names = [l.hasName for l in locations_nearby]
    items_nearby = current_location.INDIRECT_containsItem

    # test the prompt generation
    print(prompt.invoke({'setting': setting,
                    'locations': locations,
                    'characters_nearby': characters_nearby,
                    'items_nearby': items_nearby,
                    'player': player,
                    'player_start_location_name': current_location.hasName,
                    'nearby_locations_names': nearby_locations_names,
                    'language': language}).text)

chain = prompt | model
adventure_start = chain.invoke({'setting': setting,
                    'locations': locations,
                    'characters_nearby': characters_nearby,
                    'items_nearby': items_nearby,
                    'player': player,
                    'player_start_location_name': current_location.hasName,
                    'nearby_locations_names': nearby_locations_names,
                    'language': language})


Parent run 16c3da0d-43a6-4028-b65a-09a10d249cf9 not found for run 5bade0a9-bd6c-4178-9425-cf820ef3e111. Treating as a root run.



You are an LLM designed to act as the engine for a text adventure game set in "Normandie viking du 9e siècle".

List of the locations that make up this game's world (to give you general context):
    - Village de Gravepine: Un village normand typique entouré d'une palissade de bois. C'est ici que commence l'aventure du protagoniste, un endroit simple mais solide où vivent des familles de pêcheurs et de fermiers. Il y a aussi plusieurs bâtiments importants comme la forge locale et le temple dédié aux dieux nordiques. Ce lieu jouxte deux autres lieux clés : le Hall des Jarls et le Port de Gravepine.
    - Hall des Jarls: L'impressionnant hall situé juste hors du Village de Gravepine, c'est là que réside le grand jarl local. Décoré de trophées de bataque et de runes magiques gravées dans les murs, cet espace sert également de place publique pour discuter affaires et partager repas après chaque raid réussi.
    - Port de Gravepine: Point stratégique vital pour toute expédition maritime de

In [54]:
print(adventure_start.content)

**Bienvenue à Gravepine**

Tu te trouves debout devant le village de Gravepine, sous le soleil levant qui éclaire les toits de chaume et les murs de bois épais. Le village est paisible, avec quelques habitants qui s'éveillent lentement, tandis que les chiens bercent leurs maîtres. Tu es Erik Stormbringer, un jeune Viking ambitieux originaire de ce petit coin de Normandie. Ton objectif est de gagner la reconnaissance nécessite pour diriger ta propre flotte et conquérir un nouveau territoire.

Thora la Forgeonne, la plus grande forgeresse du village, te regarde depuis la fenêtre de sa forge, ses yeux brillants de curiosité. Elle te fait signe de venir lui parler. Et puis, tu vois le grand hall des Jarls, majestueusement décoré de trophées de bataille et de runes magiques, attirer l'attention de tous les habitants du village.

Le vent apporte l'odeur des poissons frais du port de Gravepine, où les navigateurs sont déjà en train de préparer leurs embarcations pour les prochaines croisières

In [56]:
CHAT_SYSTEM_PROMPT = """
You are an LLM designed to act as the engine for a text adventure game set in "{{setting}}".

Keep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.
Keep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large.

The game is played by interactions between the game (you) and the player. The player will type in natural language whatever they want to do, etc.
Do not reveal everything all at once: let the player discover things. Only output natural text, as one would read in a book they actually were the hero of.

Your messages should be short. Please do not produce lengthy messages. Your messages should be one to two sentences long. The player can always ask for more details ! For dialogues, you will output only one line of dialogue, and let the player respond.

Use the game elements provided.

Always answer in {{language}}
"""

In [58]:
HUMAN_MESSAGE_TEMPLATE = """
[INST]
The player's current location is "{{location.hasName}}". {{location.hasName}} is described as "{{location.hasDescription}}".

The locations accessible from where the player is are {{nearby_locations_names}}. You should discreetly tell the player they can go there, without being too explicit.

The player's character is named "{{player.hasName}}" and is described as "{{player.hasDescription}}". The player's goal is "{{player.hasGoal.hasDescription}}"

Characters present in {{location.hasName}}:
{%- for character in characters_nearby %}
    - {{ character.hasName }}: {{character.hasDescription}} (narrative importance {{character.hasImportance}})
{%- endfor %}

Items present in {{location.hasName}}:
{%- for item in items_nearby %}
    - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
{%- endfor %}

Answer directly and briefly to the user's message. You answer should be one or two sentences at most ! If the player is curious they will ask.
[/INST]
The player's message:
{{message}}
"""

In [59]:


## Noooo this is the old way... The new way is intolerably complex.
# https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

# This is definitely overkill for my use....
# Should I just ditch langchain ?


from typing import Optional, List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory



# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []



chat_prompt = ChatPromptTemplate.from_messages([
    ("system", CHAT_SYSTEM_PROMPT),
    ('ai', adventure_start.content),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{{message}}"),
], template_format='jinja2')

chain = chat_prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="message",
    history_messages_key="history",
)


# Retrieve relevant information from the KG
with onto:
    player = list(onto.Player.instances())[0]
    current_location = player.INDIRECT_isAtLocation
    characters_nearby = current_location.INDIRECT_containsCharacter
    locations_nearby = current_location.INDIRECT_isLinkedToLocation
    nearby_locations_names = [l.hasName for l in locations_nearby]
    items_nearby = current_location.INDIRECT_containsItem


    print(chain_with_history.invoke(  # noqa: T201
        {"setting": setting, 
        "language": language, 
        'characters_nearby': characters_nearby,
        'items_nearby': items_nearby,
        'player': player,
        'nearby_locations_names': nearby_locations_names,
        "message": "Aller à la forge"},
        config={"configurable": {"session_id": "foo"}}
    ).content)

# Uses the store defined in the example above.
print(store)  # noqa: T201

# print(chain_with_history.invoke(  # noqa: T201
#     {"setting": setting, "question": "What's its inverse"},
#     config={"configurable": {"session_id": "foo"}}
# ))

# print(store)  # noqa: T201

Parent run 37d56499-8f7d-4329-9325-0b700467d53b not found for run 70f776f7-1b3a-4026-b6dd-64cac8ebc716. Treating as a root run.


[llm/start] [llm:ChatMLX] Entering LLM run with input:
{
  "prompts": [
    "System: \nYou are an LLM designed to act as the engine for a text adventure game set in \"Normandie viking du 9e siècle\".\n\nKeep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.\nKeep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large.\n\nThe game is played by interactions between the game (you) and the player. The player will type in natural language whatever they want to do, etc.\nDo not reveal everything all at once: let the player discover things. Only output natural text, as one would read in a book they actually were the hero of.\n\nYour messages should be short. Please do not produce lengthy messages. Your messages should be one to two sentences long. The player can a

In [60]:
player_message = "Thora, je compte partir conquerrir des terres. Va tu m'apporter ton soutien ?"

with onto:
    player = list(onto.Player.instances())[0]
    current_location = player.INDIRECT_isAtLocation
    characters_nearby = current_location.INDIRECT_containsCharacter
    locations_nearby = current_location.INDIRECT_isLinkedToLocation
    nearby_locations_names = [l.hasName for l in locations_nearby]
    items_nearby = current_location.INDIRECT_containsItem


    print(chain_with_history.invoke(  # noqa: T201
        {"setting": setting, 
        "language": language, 
        'characters_nearby': characters_nearby,
        'items_nearby': items_nearby,
        'player': player,
        'nearby_locations_names': nearby_locations_names,
        "message": player_message},
        config={"configurable": {"session_id": "foo"}}
    ).content)

Parent run 63e1d314-8406-4a33-bece-60875db6a5c7 not found for run cbe6685b-7c43-48e4-b032-22b3f603f795. Treating as a root run.


[llm/start] [llm:ChatMLX] Entering LLM run with input:
{
  "prompts": [
    "System: \nYou are an LLM designed to act as the engine for a text adventure game set in \"Normandie viking du 9e siècle\".\n\nKeep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.\nKeep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large.\n\nThe game is played by interactions between the game (you) and the player. The player will type in natural language whatever they want to do, etc.\nDo not reveal everything all at once: let the player discover things. Only output natural text, as one would read in a book they actually were the hero of.\n\nYour messages should be short. Please do not produce lengthy messages. Your messages should be one to two sentences long. The player can a

So, we will pre-generate the first AI message, which lays out the start of the game.

Then, all subsequent turns will use the same system prompt, as well as dynamically enhancing the player's message with relevant information from the KG

After the player sends their message, we either can:

- Have the LLM respond without knowledge, then analyze the message pair for changes to the KG
- Have the LLM analyze the user's message for intent (especially moving to new locations, which will require the information of the new location to generate the response)

I think the second approach is just better, especially in the case of moving to new locations, which change the knowledge necessary to generate the response.



Once the AI response is generated, we also want to extract changes to the world from the message pair, based on the AI response (item lost, item gained)


So... Basically we need to do intent analysis of the Human message.


Let's summarize a typical conversation turn:

0. The player reads the game's message and decides on something to do.
1. The player's message is analyzed with a prompt, made to extract actionable intent that requires new knowledge (move to other location)
2. If the player moves to another location, we need to retrieve the information about this new location and use that info to generate the AI response. At the same time, we update the KG (player's position)
3. Generate the game response
4. While the player reads and decides on their next turn and types it, analyze the last (message, response) pair for changes to the world: updates to the player's inventory (remove, add or change item's description), updates to characters.

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class InMemoryHistoryWithSummary(BaseChatMessageHistory, BaseModel):
    """In-memory implementation of chat message history with a running summary."""
    
    messages: List[BaseMessage] = Field(default_factory=list)
    k: int = Field(..., ge=1)  # Max length of the message history
    current_summary: str = Field(default="")  # Current summary of all interactions

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store and update the summary if necessary."""
        for message in messages:
            # Add the message to the history
            self.messages.append(message)
            
            # If the length exceeds the max length (k), summarize and discard the oldest message
            if len(self.messages) > self.k:
                discarded_message = self.messages.pop(0)  # Remove the oldest message
                self.current_summary = self.summarize(self.current_summary, discarded_message.content)
    
    def summarize(self, current_summary: str, discarded_message: str) -> str:
        """Summarize the history, discarding the content of the oldest message."""
        # Here you could implement any logic to create a summary. This example just appends the discarded message to the summary.
        # This is where you could call an external model or summarization function.
        return f"{current_summary} [Discarded Message: {discarded_message}]"
    
    def clear(self) -> None:
        """Clear the message history and reset the summary."""
        self.messages = []
        self.current_summary = ""
